<a href="https://colab.research.google.com/github/mdjamina/m1_ml_lang_detector/blob/main/ML_lang_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Corpus
[Tatoeba](https://tatoeba.org/fr/downloads
) est une collection de phrases et de traductions.

Extraction de l'archive téléchargée 

In [17]:
!unzip ../data/data.zip

Archive:  ./data/data.zip
warning [./data/data.zip]:  zipfile claims to be last disk of a multi-part archive;
  attempting to process anyway, assuming all parts have been concatenated
  together in order.  Expect "errors" and warnings...true multi-part support
  doesn't exist yet (coming soon).
file #1:  bad zipfile offset (local header sig):  4


## Chargement du corpus

In [36]:
import pandas as pd


data = pd.read_pickle("../data/data.pkl") 


data.shape

,content,lang
0,我們試試看！,zho
1,我该去睡觉了。,zho
2,你在干什麼啊？,zho
3,這是什麼啊？,zho
4,今天是６月１８号，也是Muiriel的生日！,zho
...,...,...
10262652,Utálom a melóm!,hun
10262653,Ich habe den Ersatzschlüssel verloren.,deu
10262654,Mibe' fogadjunk?,hun
10262655,Utálok itt lenni!,hun


# pre-processing

## TODO

In [16]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data['content'], data['lang'], test_size=0.2, train_size=0.50, random_state=1) 
# ramdom_state = meme corpus (meme decoupage, decouper en 2 train et test le corpus, car on a besoin de faire les tests)

In [ ]:
y_train.value_counts()

# Model

In [4]:
import re
def ngram_2c(text):
  #text = text.replace(' ','-')
  return [re.sub('[ ]?','',i+j) for (i,j) in zip(*[text[i:] for i in range(2)])]
  #ng = [re.sub('[ ]?','',i+j+k) for (i,j,k) in zip(*[text[i:] for i in range(3)])]
  #ng = [i+j+k+m for (i,j,k,m) in zip(*[text[i:] for i in range(4)]) if j!=' ' or k!=' ']
  #for (l,m) in zip(*[text[i:] for i in range(2)]):
   # gram=remove_punctuation(l+m)



def char_tokenizer(text):
  """Character Tokenization splits apiece of text 
  into a set of characters.
  """

  return [c for c in text if c not in ' \t\n']



In [ ]:
char_tokenizer('un test de text 그러나 주로 개인적인 용도로 이용되는 위키도 있는데, 이를 개인 위키라고 한다')

In [6]:
from joblib import Memory
from shutil import rmtree

# Create a temporary folder to store the transformers of the pipeline
mem = Memory(location="/tmp/cachedir", verbose=0)

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(tokenizer=to_char, ngram_range=(1,2))

x_train_counts = count_vect.fit_transform(x_train)
x_train_counts.shape

(896115, 108154)

In [ ]:
x_train_counts
#278559

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

model = Pipeline([('TfidfVectorizer', TfidfVectorizer(tokenizer=char_tokenizer, ngram_range=(1,2),)),
                     ('LinearSVC', LinearSVC()),],verbose=True,memory=mem)


model.fit(x_train, y_train)  




[Pipeline] ... (step 1 of 2) Processing TfidfVectorizer, total= 1.4min


/home/amina/.local/lib/python3.9/site-packages/sklearn/pipeline.py:348: UserWarning: Persisting input arguments took 16.57s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(


[Pipeline] ......... (step 2 of 2) Processing LinearSVC, total= 8.5min


Pipeline(memory=Memory(location=/tmp/cachedir/joblib),
         steps=[('TfidfVectorizer',
                 TfidfVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function char_tokenizer at 0x7fc8fe94b280>)),
                ('LinearSVC', LinearSVC())],
         verbose=True)

In [18]:
model[0].get_feature_names_out().shape

(242149,)

In [19]:
predictions = model.predict(x_test)

In [7]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[ 13733     11      0 ...      0      0      0]
 [     0 160373      0 ...      0      0      0]
 [     0      0   5154 ...    277      0      0]
 ...
 [     0      0     66 ...  53881      0      0]
 [     0      1      0 ...      0   6901      0]
 [     0      0      0 ...      0      0  28047]]


In [23]:
from sklearn import metrics
print( metrics.classification_report(y_test,predictions, output_dict=False))

              precision    recall  f1-score   support

         ara       1.00      0.99      0.99      8335
         ber       0.81      0.82      0.81    118364
         bul       0.92      0.68      0.78      4828
         ces       0.94      0.95      0.95     12616
         dan       0.82      0.84      0.83     10837
         deu       0.98      0.98      0.98    115912
         ell       1.00      1.00      1.00      6972
         eng       0.98      0.99      0.99    314597
         est       0.92      0.55      0.69       769
         fas       0.99      0.99      0.99      4975
         fin       0.98      0.97      0.97     27681
         fra       0.98      0.97      0.97     99984
         gle       0.98      0.89      0.93       533
         hbs       0.91      0.83      0.87     10239
         heb       1.00      1.00      1.00     39775
         hin       1.00      1.00      1.00      2861
         hun       0.99      0.99      0.99     70925
         hye       0.97    

In [15]:
rmetrics

{'ara': {'precision': 0.9952346914462712,
  'recall': 0.9905145838273655,
  'f1-score': 0.9928690278107917,
  'support': 4217},
 'ber': {'precision': 0.8043758469114585,
  'recall': 0.8228273915174874,
  'f1-score': 0.8134970042867529,
  'support': 59157},
 'bul': {'precision': 0.9251276233692569,
  'recall': 0.6613949716139497,
  'f1-score': 0.7713407424923149,
  'support': 2466},
 'ces': {'precision': 0.9475352670787763,
  'recall': 0.9426048565121413,
  'f1-score': 0.9450636313334915,
  'support': 6342},
 'dan': {'precision': 0.8262842621165366,
  'recall': 0.8350761328196661,
  'f1-score': 0.8306569343065694,
  'support': 5451},
 'deu': {'precision': 0.979904520621133,
  'recall': 0.9839745253794369,
  'f1-score': 0.9819353055972919,
  'support': 57783},
 'ell': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3485},
 'eng': {'precision': 0.9813668511491282,
  'recall': 0.989179675835033,
  'f1-score': 0.9852577753448237,
  'support': 157389},
 'est': {'precision': 0.9

In [21]:
metrics.accuracy_score(y_test,predictions)

0.9510503674193602

In [12]:
model.predict(['그러나 주로 개인적인 용도로 이용되는 위키도 있는데, 이를 개인 위키라고 한다. bonjour'])[0] 

'kor'

In [25]:
model.predict(['ויקי יכולה להיות שיטה טובה לשיתוף ידע בקהילות שפועלות באמצעות האינטרנט או בתוךיות. היא חלופה'])[0]

'heb'

In [11]:
model.predict(['einsetzt. Zudem nutzen auch viele '])[0]

'deu'

In [11]:
model.predict(['الكرام بتحويل الكتابة العربي الى كتابة مفهومة من قبل اغلب برامج التصميم مثل الفوتوشوب و الافترايفكتس و البريمير و الافد ميدا كومبوزر و السموك و برامج اخرى'])[0]

'ara'

In [35]:
model.predict(
    ['से प्रत्येक एक विशिष्ट भाषा से संबंधित है। विकिपीडिया के अलावा, सार्वजनिक और निजी दोनों उपयोग में सैकड़ों हजारों अ'
    ,'Store metadata, particularly for linguistic annotations of a certain type'
    ,'bonjour et body are parfois c''est logique état'])

array(['hin', 'eng', 'fra'], dtype=object)

# Model persistence

In [22]:
from joblib import dump
dump(model, '../data/lang_detector.joblib') 

['data/lang_detector.joblib']